# Precog Project
## Image Recognition Part

In [1]:
#  I have a folder in which named clean_dataset, which has 3 sub folder named namo, ak and people 
#  containing 50 images in each of them.

In [ ]:
#  Basic Idea about problem is that is looks like typical Classification Problem, 
# Where classes can be namely NAMO, AK, People

In [6]:
import sys
import os
from skimage import io
# import dlib
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
import pymongo
from bson import Binary
import bson
from bson.binary import Binary as BsonBinary
import numpy as np
from PIL import Image
from scipy import ndimage
from six.moves import cPickle as pickle
from sklearn.metrics import accuracy_score
np.random.seed(133)

In [28]:
connection = MongoClient()
db = connection['mongo_database']

In [7]:
# We now want to convert the entire dataset into a 3D array
# (image index, x, y), with mean = 0  and s.d. = 0.5
#  give 3 diff folderpaths namo, ak, people
img_wd = 200 # resized Images
img_ht = 200 # resized Images

def load_pictures(folderpath):
    '''Loading for a single image'''
    img_files = os.listdir(folderpath)
    tot_imgs = len(img_files)

#     creating dataset for images of each class, dataset will be of size [totalimages * (img_wd*img_ht*3 =(600))]
    dataset = np.ndarray(shape = (tot_imgs, img_wd*img_ht*3),
                         dtype = np.float32)
    img_index = 0
    for img in img_files:
        img_file_path = os.path.join(folderpath, img)
        try:
            img_data = (ndimage.imread(img_file_path).astype(float) - 255.0/2)/255.0
#              We did the above to set the range of the image data to 
#             -0.5 to 0.5 (s.d.)

#             reshaping a (200,200,3) matrix to (12000), flattening
            img_data = img_data.reshape(img_wd*img_ht*3)
    
#     adding flattened img_data to dataset
            dataset[img_index, :] = img_data
            img_index = img_index + 1
        except IOError as e:
            print('file read error, skipping thsi file')
                
    print('Full dataset shape', dataset.shape)
    print('Dataset Mean', np.mean(dataset))
    print('Dataset SD', np.std(dataset))
    return dataset

In [8]:
data_set_ak = load_pictures("./clean_dataset/ak")
data_set_namo = load_pictures("./clean_dataset/namo")
data_set_people = load_pictures("./clean_dataset/people")

/home/sharma/.local/lib/python2.7/site-packages/ipykernel_launcher.py:19: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0.
Use ``matplotlib.pyplot.imread`` instead.


('Full dataset shape', (400, 120000))
('Dataset Mean', -0.06528978)
('Dataset SD', 0.24625796)
('Full dataset shape', (400, 120000))
('Dataset Mean', 0.0063913618)
('Dataset SD', 0.22900325)
('Full dataset shape', (400, 120000))
('Dataset Mean', -0.030012265)
('Dataset SD', 0.2670311)


In [9]:
# now i have created Flattened Image data set for each three 
# of categories

#  one such db can be 
# [ R,G,B,R,G,B,R,G,B,R,G,B,R,G,B,R,G,B ..... 200*200  ] pixals
# .
# .
# .
# .
# .
# for all images for a given person
# so now size of a given db is (400*20000)


In [10]:
# now I have to divide dataset in TRAIN, TEST, CROSS_VALIDATION
# total 400 images of each
# 50% --> TRAIN (300 imgs in each class)
# 25% -->TEST (50 imgs '' )
# 25% -->CV (50 imgs '')
# CV = Cross Validation Data


In [11]:
dataset_train_namo = data_set_namo[0:300]
dataset_test_namo = data_set_namo[300:350]
dataset_cv_namo = data_set_namo[350:400]

dataset_train_ak = data_set_ak[0:300]
dataset_test_ak = data_set_ak[300:350]
dataset_cv_ak = data_set_ak[350:400]

dataset_train_people = data_set_people[0:300]
dataset_test_people = data_set_people[300:350]
dataset_cv_people = data_set_people[350:400]


In [12]:
dataset_train_final = np.concatenate((dataset_train_namo,dataset_train_ak))
dataset_train_final = np.concatenate((dataset_train_final,dataset_train_people))
print dataset_train_final.shape

dataset_test_final = np.concatenate((dataset_test_namo,dataset_test_ak))
dataset_test_final = np.concatenate((dataset_test_final,dataset_test_people))
print dataset_test_final.shape

dataset_cv_final = np.concatenate((dataset_cv_namo,dataset_cv_ak))
dataset_cv_final = np.concatenate((dataset_cv_final,dataset_cv_people))
print dataset_test_final.shape


(900, 120000)
(150, 120000)
(150, 120000)


In [13]:
#  next task to create a label for each dataset 
# if 1 represents modi
# it should be like
# [1]
# [1]
# [1]
# [1]

# CONVENTION -
# 1 --> NAMO
# 2 --> AK
# 3 --> PEOPLE

In [14]:
# total row in train data set = 900

label_train = np.ndarray(900, dtype=np.int32)
label_train[0:300]=1 # for namo 
label_train[300:600]=2 # for ak
label_train[600:900]=3 # for people
print label_train.size

label_test = np.ndarray(150, dtype=np.int32)
label_test[0:50]=1 # for namo 
label_test[50:100]=2 # for ak
label_test[100:150]=3 # for people
print label_test.size


label_cv = np.ndarray(150, dtype=np.int32)
label_cv[0:50]=1 # for namo 
label_cv[50:100]=2 # for ak
label_cv[100:150]=3 # for people
print label_cv.size

#     just an other way to label, I was trying
# label_array_namo = np.full((400,1),1)
# label_array_ak = np.full((400,1),2)
# label_array_people = np.full((400,1),3)
# # print label_array_ak

900
150
150


In [15]:

# both dataset and labels are arranged w.r.t. variable perm 

def randomize(dataset,labels,size):
    perm = np.random.permutation(size)
    shuffled_dataset = dataset[perm,:]
    shuffled_labels = labels[perm]
    return shuffled_dataset, shuffled_labels

In [16]:
dataset_train_final, label_train = randomize(dataset_train_final,label_train,900)
dataset_test_final, label_test = randomize(dataset_test_final,label_test,150)
dataset_cv_final, label_cv = randomize(dataset_cv_final,label_cv,150)

print dataset_train_final.shape,label_train.shape
print dataset_test_final.shape, label_test.shape
print dataset_cv_final.shape, label_cv.shape
# A sample printout, to be sure of ramdomised data
print label_test


(900, 120000) (900,)
(150, 120000) (150,)
(150, 120000) (150,)
[1 3 3 3 3 3 2 2 1 2 1 1 3 3 1 3 3 2 3 2 3 3 3 3 2 2 3 1 1 2 2 1 1 1 2 2 1
 1 2 1 2 2 1 1 2 2 1 2 3 2 3 3 1 3 2 1 1 3 1 3 3 2 2 1 3 2 3 2 1 3 1 3 3 2
 2 1 1 2 3 3 1 3 2 2 1 2 1 1 2 2 3 1 1 3 2 1 2 1 2 3 1 1 2 2 3 3 3 3 1 1 3
 2 1 2 2 2 1 2 1 3 2 3 3 1 2 1 3 1 1 1 3 3 2 1 2 2 3 1 3 3 2 1 3 3 2 2 1 1
 2 3]


In [71]:
# saving files to mongoDB
#  THE ERROR I COULD NOT SAVE FILES IN mongoDB
#  mongoDB only allows file upto size 16MB
# this works, YAY!! \/,
# db.insert({'dataset_train_final': Binary(pickle.dumps(dataset_train_final))})

db['dataset_train_final'].insert({'dataset_train_final': Binary(pickle.dumps(dataset_train_final))})
db['label_train'].insert({'label_train': Binary(pickle.dumps(label_train))})
db['dataset_test_final'].insert({'dataset_test_final': Binary(pickle.dumps(dataset_test_final))})
db['label_test'].insert({'label_test': Binary(pickle.dumps(label_test))})
db['dataset_cv_final'].insert({'dataset_cv_final': Binary(pickle.dumps(dataset_cv_final))})
db['label_cv'].insert({'label_cv': Binary(pickle.dumps(label_cv))})

/home/sharma/.local/lib/python2.7/site-packages/ipykernel_launcher.py:6: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  


DocumentTooLarge: BSON document too large (1484657743 bytes) - the connected server supports BSON document sizes up to 16793598 bytes.

In [18]:
# saving files to .pickle

with open("./dataset_train_final.pickle",'wb') as f:
    pickle.dump(dataset_train_final,f, pickle.HIGHEST_PROTOCOL)

with open("./label_train.pickle",'wb') as f:
    pickle.dump(label_train,f, pickle.HIGHEST_PROTOCOL)

with open("./dataset_test_final.pickle",'wb') as f:
    pickle.dump(dataset_test_final,f, pickle.HIGHEST_PROTOCOL)

with open("./label_test.pickle",'wb') as f:
    pickle.dump(label_test,f, pickle.HIGHEST_PROTOCOL)

with open("./dataset_cv_final.pickle",'wb') as f:
    pickle.dump(dataset_cv_final,f, pickle.HIGHEST_PROTOCOL)

with open("./label_cv.pickle",'wb') as f:
    pickle.dump(label_cv,f, pickle.HIGHEST_PROTOCOL)


In [19]:
# for fetching data from mongoDB 
#  Could not use it, beacause unable to save in mongoDB
# dataset_train_final = [np.array(x['dataset_train_final']) for x in db['dataset_train_final'].find()]
# label_train = [np.array(x['label_train']) for x in db['label_train'].find()]
# dataset_test_final = [np.array(x['dataset_test_final']) for x in db['dataset_test_final'].find()]
# label_test = [np.array(x['label_test']) for x in db['label_test'].find()]
# dataset_cv_final = [np.array(x['dataset_cv_final']) for x in db['dataset_cv_final'].find()]
# label_cv = [np.array(x['label_cv']) for x in db['label_cv'].find()]

In [20]:
#  for fetching data from .pickle files

with open("./dataset_train_final.pickle", 'rb') as f:
    dataset_train_final = pickle.load(f)

with open("./label_train.pickle", 'rb') as f:
    label_train = pickle.load(f)

with open("./dataset_test_final.pickle", 'rb') as f:
    dataset_test_final = pickle.load(f)

with open("./label_test.pickle", 'rb') as f:
    label_test = pickle.load(f)

with open("./dataset_cv_final.pickle", 'rb') as f:
    dataset_cv_final = pickle.load(f)

with open("./label_cv.pickle", 'rb') as f:
    label_cv = pickle.load(f)


### Training Begins     \m/    (0_0)

In [ ]:
# Now, after setting up dataset, I need to train it

In [23]:
from sklearn.svm import SVC

In [24]:
from sklearn.linear_model import LogisticRegression

In [25]:
model = LogisticRegression()

In [35]:
model.fit(dataset_train_final,label_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [36]:
# saving the Logistic_regression model object, for further,so we don't need to train
# again and again.

with open("./Logistic_regression_fit.pickle",'wb') as f:
    pickle.dump(model,f, pickle.HIGHEST_PROTOCOL)

In [37]:
with open("./Logistic_regression_fit.pickle", 'rb') as f:
    model_from_pickle = pickle.load(f)

In [38]:
predicc = model_from_pickle.predict(dataset_test_final)

In [39]:
print(accuracy_score(predicc,label_test))

0.946666666667


In [40]:
#  running it on cross validation

In [41]:
predicc = model_from_pickle.predict(dataset_cv_final)

In [42]:
print(accuracy_score(predicc,label_cv))

0.953333333333


In [ ]:
# yay!!!, got a accuracy of about 95.3%., :D Logistic Regression works like Charm

In [ ]:
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [14]:
#  Let's try with SVM classifer. 

In [68]:
classifier = SVC()
print X_shuf.shape
print Y_shuf.shape

(900, 120000)
(900,)


In [82]:
classifier.fit(X_shuf,Y_shuf)
classifier.fit(dataset_train_final,label_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [83]:
predicc = classifier.predict(dataset_test_final)

In [86]:
print(accuracy_score(predicc,label_test))

0.3


In [ ]:
#  So i got a accuracy of 30% with SVM.

In [ ]:
# Logistic Regression is used to ascertain the probability of an event. 
# And this event is captured in binary format, i.e. 0 or 1.

In [ ]:
# So it comes out SVM's are not good if features are quite large(120000) and Logistic Regression outperforms it.